In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

In [2]:
import requests
# json 라이브러리가 아닌 xml 라이브러리를 이용해서 xml 파싱을 해보자.
from xml.etree.ElementTree import fromstring, ElementTree
from elasticsearch import Elasticsearch
es = Elasticsearch('http://localhost:9200/')

한글 분석기와 위경도 데이터 타입을 이용해야 하기 때문에 파이썬 코드를 실행 전에 인덱스 매핑을 먼저 실행한다.  
place_nm은 데이터가 한글이고 한글의 경우 엘라스틱서치에서 제공하는 nori 한글 형태소 분석기를 사용해야 전문 검색이 제대로 수행된다.  
엘라스틱서치가 설치된 폴더에서 아래의 명령을 실행해 nori 형태소 분석기를 설치한다.  
.\bin\elasticsearch-plugin install analysis-nori  
nori 형태소 분석기를 설치했으면 settings에서 설정을 해야한다.  
와이파이 설치 위치인 위경도 데이터는 geo_point 데이터 타입으로 설정한다.

&nbsp;PUT seoul_wifi  
&nbsp;{  
&nbsp;&nbsp;"settings": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"analysis": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"analyzer": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"korean": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"tokenizer": "nori_tokenizer"  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;}  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;}  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;}  
&nbsp;&nbsp;&nbsp;},   
&nbsp;&nbsp;&nbsp;"mappings": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"properties": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"gu_nm": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"type": "keyword"  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;},  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"place_nm": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"type": "text",  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"analyzer": "korean"  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;},  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"geo_xy": {  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"type": "geo_point"  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;}  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;}  
&nbsp;&nbsp;&nbsp;}  
&nbsp;}

In [3]:
# 2023년 11월 14일 현재 공공 와이파이 개수는 23488개 인데 오픈 API에서는 한 번에 호출할 수 있는 개수가 제한되서
# 1000개씩 나눠서 API를 호출한다.
for i in range(1, 25)[:]:
    iStart = (i - 1) * 1000 + 1
    iEnd = i * 1000
    # print('{:5d}, {:5d}'.format(iStart, iEnd))
    # 오픈 API 요청 주소를 만든다.
    apiRequest = 'http://openapi.seoul.go.kr:8088/746e4968796d6b7639366763624a53/xml/TbPublicWifiInfo/{}/{}/'.format(iStart, iEnd)
    # apiRequest = 'http://openapi.seoul.go.kr:8088/746e4968796d6b7639366763624a53/xml/TbPublicWifiInfo/1/2/'
    # print(apiRequest)
    
    response = requests.get(apiRequest) # 오픈 API를 요청한다.
    tree = ElementTree(fromstring(response.text)) # 오픈 API가 응답한 내용을 xml로 파싱한다.
    # print(i, tree)
    root = tree.getroot() # 파싱된 xml의 최상단의 root 태그(TbPublicWifiInfo)를 얻어온다.
    # print(root)
    
    # 파싱된 xml의 <row> ~ </row> 사이의 모든 데이터를 반복을 수행하며 얻어온다.
    for row in root.iter('row'):
        # print(row)
        # <row> ~ </row> 사이의 데이터 중에서 엘라스틱서치에 저장할 데이터만 가져온다.
        gu_nm = row.find('X_SWIFI_WRDOFC').text # 구이름
        place_nm = row.find('X_SWIFI_MAIN_NM').text # 설치장소
        place_x = row.find('LAT').text # 경도, x축
        place_y = row.find('LNT').text # 위도, y축
        # print(gu_nm, place_nm, place_x, place_y)
        
        # <row> ~ </row> 사이에서 엘라스틱서치에 저장하기 위해 가져온 데이터를 딕셔너리 형태로 만든다.
        # geo_point 타입에는 위도(y축), 경도(x축)의 순서로 데이터를 구성해야 한다.
        doc = {
            'gu_nm': gu_nm,
            'place_nm': place_nm,
            'geo_xy': {
                'lat': float(place_y), # 위도
                'lon': float(place_x)  # 경도
            }
        }
        # print(doc)
        
        # 엘라스틱서치에 인덱싱 한다.
        try:
            response = es.index(index='seoul_wifi', body=doc)
        except:
            doc = {
                'gu_nm': gu_nm,
                'place_nm': place_nm,
                'geo_xy': {
                    'lat': float(place_x),
                    'lon': float(place_y)
                }
            }
            print(doc)
            response = es.index(index='seoul_wifi', body=doc)
    # ===== for
    print('종료: {:5d}, {:5d}'.format(iStart, iEnd))
print('전체종료')

종료:     1,  1000
종료:  1001,  2000
{'gu_nm': '동대문구', 'place_nm': '명성경로당', 'geo_xy': {'lat': 37.57435, 'lon': 127.03191}}
{'gu_nm': '동대문구', 'place_nm': '신설동경로당', 'geo_xy': {'lat': 37.57475, 'lon': 127.02727}}
{'gu_nm': '동대문구', 'place_nm': '신일경로당', 'geo_xy': {'lat': 37.571312, 'lon': 127.0308}}
{'gu_nm': '동대문구', 'place_nm': '용두경로당', 'geo_xy': {'lat': 37.57845, 'lon': 127.02861}}
{'gu_nm': '동대문구', 'place_nm': '만수경로당', 'geo_xy': {'lat': 37.583946, 'lon': 127.034225}}
{'gu_nm': '동대문구', 'place_nm': '제기경로당', 'geo_xy': {'lat': 37.58571, 'lon': 127.03946}}
{'gu_nm': '동대문구', 'place_nm': '행복마을경로당', 'geo_xy': {'lat': 37.589996, 'lon': 127.03865}}
{'gu_nm': '동대문구', 'place_nm': '전농1동경로당', 'geo_xy': {'lat': 37.578793, 'lon': 127.052124}}
{'gu_nm': '동대문구', 'place_nm': '창방경로당', 'geo_xy': {'lat': 37.578136, 'lon': 127.050545}}
{'gu_nm': '동대문구', 'place_nm': '청송경로당', 'geo_xy': {'lat': 37.578648, 'lon': 127.05552}}
{'gu_nm': '동대문구', 'place_nm': '답십리3동경로당', 'geo_xy': {'lat': 37.57397, 'lon': 127.04533}}
{'gu